# PokeProject


## Calculations



For Attack, Defense, Sp.Atk, Sp.Def,

$Stat = ((Base \times 2 + IV + (EV/4)) \times \frac{Level}{100} + 5) \times Nature$

For the purposes of this assignment we use IV and EV values of 0 and assume a level of 100 with a neutral Nature (x1) which simplifies the equation,

$Stat = Base \times 2 + 5$

For HP,

$HP = ((Base \times 2 + IV + (EV/4)) \times \frac{Level}{100} + 10 + Level$

For the purposes of this assignment we use IV and EV values of 0 and assume a level of 100 with a neutral Nature (x1) which simplifies the equation,

$HP = Base \times 2 + 110$

Or,

$HP = Stat(HP) + 105$


Our damage calculation is as follows:

$Damage = 	(\frac {(\frac{2 \times Level}{5} + 2) \times Power \times \frac{Attack}{Defense}}{50} + 2) \times Modifier$

For the purposes of this assignment we assume a level of 100, a power of 100, and Modifier is only type advantage, which simplifies the equation,

$Damage = 	\big(84 \times \frac{Attack}{Defense} + 2) \times Type Advantage$

Substituting in the stat calculation...

$Damage = 	\big(84 \times \frac{AttackBase \times 2 + 5}{DefenseBase \times 2 + 5} + 2) \times TypeAdvantage$

In [1]:
import pandas as pd
import numpy as np
import advantage
from advantage import TypeAdvantage
from advantage import types
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import math
import random
from IPython.display import clear_output

# Data
https://www.kaggle.com/abcsds/pokemon#

Our data contains all 721 Pokemon from the six generations of Pokemon. Each Pokemon also has it's own number, name, type, second type (if applicable), total, HP, attack, defense, Sp. Atk, Sp. Def, speed, generation, and if it is legendary or not. 

In [35]:
pokedex = pd.read_csv("Pokemon.csv")

def getPoke(pokeid):
    return np.array(pokedex.loc[pokedex['#'] == pokeid])[0]

def getPokeByName(pokeid):
    return np.array(pokedex.loc[pokedex['Name'] == pokeid])[0]

def createTeam(team):
    arr = []
    for i in range(6):
        arr.append(getPoke(int(team[i])) if team[i].isdigit() else getPokeByName(team[i]))
    return np.array(arr)

createTeam(['Gyarados','77','23','99','342','6'])
TOTAL_POKEMON = 721
pokedex

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


In [3]:
def DisplayPokemon(team, health):
    for i in range(6):
        s = str(i) + " - "
        s += str(team[i][1])
        s += "\tHP: " + (str(health[i]) if health[i] > 0 else "FNT")
        s += " ATK: " + str(team[i][6])
        s += " DEF: " + str(team[i][7])
        s += " SP. ATK: " + str(team[i][8])
        s += " SP. DEF: " + str(team[i][9])
        s += " SPD: " + str(team[i][10])
        s += "  Type " + str(team[i][2]) + " " + (str(team[i][3]) if team[i][3] == team[i][3] else "")
        print(s)
        
def Clear():
    clear_output()

class Agent():
    def __init__(self, name):
        self.name = name
    
    #The action it should use given the current state
    def getAction(self, current, enemycurrent, team, enemyteam, health, enemyhealth):
        #Uses the attack corresponding to the higher stat (Physical if Attack, Special if Sp. Attack)
        return 0 if team[current][6] > team[current][8] else 1 
    
    def getSwitch(self, current, enemycurrent, team, enemyteam, health, enemyhealth):
        return current + 1
    
class Player(Agent):
    def __init__(self, name):
        Agent.__init__(self, name)
    
    def getAction(self, current, enemycurrent, team, enemyteam, health, enemyhealth):
        
        print("What should ", team[current][1], "do?")
        print("0 - Physical Attack")
        print("1 - Special Attack")
        print("2 - Switch Pokemon")
        print("3 - Display Pokemon")
        action = input(": ")
        while not action.isdigit() or action == "3":
            if action == "3":
                print("~~~~~~~Your Team~~~~~~~")
                DisplayPokemon(team, health)
                print("~~~~~~~Enemy Team~~~~~~~")
                DisplayPokemon(enemyteam, enemyhealth)
            else:
                print("Action ", action, " is invalid!")
            print("What should", team[current][1], "do?")
            print("0 - Physical Attack")
            print("1 - Special Attack")
            print("2 - Switch Pokemon")
            action = input(": ")
        Clear()
        return int(action) 
    
    def getSwitch(self, current, enemycurrent, team, enemyteam, health, enemyhealth):
        print("Bring out which Pokemon?")
        print("0 - ", team[0][1], " %FAINTED%" if health[0] <= 0 else "", "*CURRENT*" if current == 0 else "")
        print("1 - ", team[1][1], " %FAINTED%" if health[1] <= 0 else "", "*CURRENT*" if current == 1 else "")
        print("2 - ", team[2][1], " %FAINTED%" if health[2] <= 0 else "", "*CURRENT*" if current == 2 else "")
        print("3 - ", team[3][1], " %FAINTED%" if health[3] <= 0 else "", "*CURRENT*" if current == 3 else "")
        print("4 - ", team[4][1], " %FAINTED%" if health[4] <= 0 else "", "*CURRENT*" if current == 4 else "")
        print("5 - ", team[5][1], " %FAINTED%" if health[5] <= 0 else "", "*CURRENT*" if current == 5 else "")
        switch = input(": ")
        while (not switch.isdigit()) or health[int(switch)] <= 0 or switch == current:
            print("Invalid selection!")
            print("Bring out which Pokemon?")
            print("0 - ", team[0][1], " %FAINTED%" if health[0] <= 0 else "", "*CURRENT*" if current == 0 else "")
            print("1 - ", team[1][1], " %FAINTED%" if health[1] <= 0 else "", "*CURRENT*" if current == 1 else "")
            print("2 - ", team[2][1], " %FAINTED%" if health[2] <= 0 else "", "*CURRENT*" if current == 2 else "")
            print("3 - ", team[3][1], " %FAINTED%" if health[3] <= 0 else "", "*CURRENT*" if current == 3 else "")
            print("4 - ", team[4][1], " %FAINTED%" if health[4] <= 0 else "", "*CURRENT*" if current == 4 else "")
            print("5 - ", team[5][1], " %FAINTED%" if health[5] <= 0 else "", "*CURRENT*" if current == 5 else "")
            switch = input(": ")
        return int(switch)

In [61]:

def CompilePokemon(pokemon):
    if not isinstance(pokemon, str):
        pokemon = getPoke(pokemon)
    poke = []
    poke.append(types[pokemon[2]])
    poke.append(types[pokemon[3]] if pokemon[3] == pokemon[3] else 0)
    poke.append(pokemon[5])
    poke.append(pokemon[6])
    poke.append(pokemon[7])
    poke.append(pokemon[8])
    poke.append(pokemon[9])
    poke.append(pokemon[10])
    return poke

def CompileTeamData(team, health, current):
    teamcompiled = []
    for i in range(6):
        poke = CompilePokemon(team[i])
        poke[2] *= health[i] / (poke[2] * 2 + 110) ##Normalize health
        teamcompiled.append(poke)
    teamcompiled.append(current1)
    return teamcompiled


In [7]:
#Actions
#0 - Physical Attack
#1 - Special Attack
#2 - Switch Pokemon

def PokeBattle(team1, team2, agent1, agent2, verbose=True):
    team1 = createTeam(team1)
    team2 = createTeam(team2)
    health1 = np.array([team1[0][5], team1[1][5], team1[2][5], team1[3][5], team1[4][5], team1[5][5]]) * 2 + 110
    health2 = np.array([team2[0][5], team2[1][5], team2[2][5], team2[3][5], team2[4][5], team2[5][5]]) * 2 + 110
    current1 = current2 = 0
    won = -1
    action1=0
    action2=0
    damageswing = [0,0]
    effectiveness = []
    rounds = 0
    
    def Damage(Attacker, Defender, SP):
        return (84 * (Attacker[8 if SP else 6] * 2 + 5) / (Defender[8 if SP else 6] * 2 + 5) + 2) * TypeAdvantage(Attacker[2], Attacker[3], Defender[2], Defender[3])
    def Attack(atk):
        #global damageswing
        if atk == 1:
            damage = Damage(team1[current1], team2[current2], action1 == 1)
            effectiveness[0][current1] += damage
            effectiveness[1][current2] -= damage
            damageswing[0] = damage
            #damageswing += damage
            if verbose:
                print(team1[current1][1], "used ", "Attack" if action1 == 0 else "SP. Attack", " against", 
                      team2[current2][1], "for ", damage, " damage!")
            health2[current2] -= damage
            if health2[current2] <= 0:
                health2[current2] = 0
                if verbose: 
                    print(team2[current2][1], "has fainted!")
            return damage
        elif atk == 2:
            damage = Damage(team2[current2], team1[current1], action2 == 1)
            effectiveness[1][current2] += damage
            effectiveness[0][current1] -= damage
            damageswing[1] = damage
            #damageswing -= damage
            if verbose:
                print(team2[current2][1], "used ", "Attack" if action2 == 0 else "SP. Attack", " against", 
                      team1[current1][1], "for ", damage, " damage!")
            health1[current1] -= damage
            if health1[current1] <= 0:
                health1[current1] = 0
                if verbose: 
                    print(team1[current1][1], "has fainted!")
            return -damage #Information taken from the perspective of player
    
    def CompileData(team):
        data = []
        if team == 1:
            data.append(CompileTeamData(team1, health1, current1))
            data.append(CompileTeamData(team2, health2, current2))
            data.append(action1)
        else:
            data.append(CompileTeamData(team2, health2, current2))
            data.append(CompileTeamData(team1, health1, current1))
            data.append(action2)
        return data
                              
            
    #def 
    
    while won < 0 and rounds < 100:
        damageswing = [0,0]
        if verbose:
            print()
        action1 = agent1.getAction(current1, current2, team1, team2, health1, health2)
        action2 = agent2.getAction(current2, current1, team2, team1, health2, health1)
        if action1 == 2:
            current1 = agent1.getSwitch(current1, current2, team1, team2, health1, health2)
            if verbose: 
                print(agent1.name, "has swapped out for", team1[current1][1], "!")
        if action2 == 2:
            current2 = agent2.getSwitch(current2, current1, team2, team1, health2, health1)
            if verbose: 
                print(agent2.name, "has swapped out for", team2[current2][1], "!")
        
        if action1 != 2 and action2 != 2:
            if team1[current1][10] > team2[current2][10] or (team1[current1][10] == team2[current2][10] and bool(random.getrandbits(1))):
                Attack(1)
                if health2[current2] > 0:
                    Attack(2)
            else:
                Attack(2)
                if health1[current1] > 0:
                    Attack(1)
        
        elif action1 != 2:
            Attack(1)
        elif action2 != 2:
            Attack(2)
        
        if health1[current1] <= 0:
            if sum(health1) <= 0:
                won = 2
            else:
                current1 = agent1.getSwitch(current1, current2, team1, team2, health1, health2)
                if verbose: 
                    print(agent1.name, "has swapped out for", team1[current1][1], "!")
        if health2[current2] <= 0:
            if sum(health2) <= 0:
                won = 1
            else:
                current2 = agent2.getSwitch(current2, current1, team2, team1, health2, health1)
                if verbose: 
                    print(agent2.name, "has swapped out for", team2[current2][1], "!")
        rounds += 1
    print(agent1.name if won == 1 else agent2.name, "has won!")

In [19]:
#for i in range(20):
#    PokeBattle(['Gyarados','77','23','99','342','6'], 
#               ['Gyarados','77','23','99','342','6']
#               , Agent("AI 1"), Agent("AI 2"), verbose=False)

In [20]:
#PokeBattle(['Mewtwo','Mewtwo','Mewtwo','Mewtwo','Mewtwo','Mewtwo'], 
#           ['Mew','Mew','Mew','Mew','Mew','Mew']
#           , Agent("Brenden"), Agent("Srikar"), verbose=True)

In [9]:
def ResetTensorflow(seed):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
    
def Selu(z, scale=1.0507009873554804934193349852946, alpha=1.6732632423543772848170429916717):
    return scale * tf.where(z >= 0.0, z, alpha * tf.nn.elu(z))

In [16]:
def neuralNet(Xtrain, Xtest, layers, n_epochs=10000, learning_rate=0.01):
    ResetTensorflow(100)

    # inputs for training
    X = tf.placeholder(tf.float32, shape=(None, layers[0]))

    # neural network building 
    with tf.name_scope("nn"):
        h = [tf.layers.dense(X, layers[0])]#, activation=Selu, name="h0")]
        for i in range(1, len(layers)):
            h.append(tf.layers.dense(h[i-1], layers[i]))#, activation=Selu, name="h" + str(i)))
        outputs = tf.layers.dense(h[len(layers)-1], layers[len(layers)-1])#, name="out")

    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.square(outputs - X))

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)
        
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        init.run()

        errorTrace = []
        for epoch in range(n_epochs):
            if epoch % 1000 == 0:
                acc_train = loss.eval({X: Xtrain})
                acc_test = loss.eval({X: Xtest})
                print("Epoch", epoch, "Train =", acc_train, "Test =", acc_test)
            err, _ = sess.run([loss, training_op], {X: Xtrain})
            errorTrace.append(err)

        acc_train = loss.eval({X: Xtrain})
        acc_test = loss.eval({X: Xtest})
        print("Train Accuracy: ", acc_train)
        print("Test Accuracy: ", acc_test)

In [37]:
Xtrain = []
Xtest = []


i = 1
while i < TOTAL_POKEMON:
    Xtrain.append(CompilePokemon(i))
    i += 1
    Xtest.append(CompilePokemon(i))
    i += 1

Xtrain = np.array(Xtrain)
Xtest = np.array(Xtest)
    
#TESTING INDIVIDUAL POKEMON AUTO-ENCODING
neuralNet(Xtrain, Xtest, [8, 8], n_epochs=3000)

Epoch 0 Train = 6379.89 Test = 6135.46
Epoch 1000 Train = 2.8465 Test = 2.86171
Epoch 2000 Train = 0.00622624 Test = 0.00635779
Train Accuracy:  0.0013104
Test Accuracy:  0.00133879


In [60]:
def EnumerateTeam(team, health, current):
    teamcompiled = []
    for i in range(6):
        poke = CompilePokemon(team[i])
        poke[2] *= health[i] ##Normalize health
        teamcompiled += poke
    teamcompiled += [current]
    return teamcompiled
    
    
def EnumerateSamples(numSamples=10000):
    teams = []
    for i in range(numSamples):
        teams.append(EnumerateTeam(np.random.randint(1, TOTAL_POKEMON + 1, (6)), 
                                   np.random.random((6)), 
                                   np.random.randint(0, 6)))
    return teams
    
def EnumerateMatchups(teams):
    return

In [69]:
samples = EnumerateSamples(3000)



# 

In [71]:
Xtrain = samples[1000:]
Xtest = samples[:1000]

#TESTING POKEMON TEAM AUTO-ENCODING
neuralNet(Xtrain, Xtest, [49, 49], n_epochs=2000)

Epoch 0 Train = 5902.92 Test = 5942.8
Epoch 1000 Train = 2.47335 Test = 2.61743
Train Accuracy:  0.258106
Test Accuracy:  0.286838
